# **LLM Serving with Apigee**

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/apigee-samples/blob/main/llm-logging/llm_logging_v1.ipynb">
          <img src="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/images/icon32.png?raw=true" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fapigee-samples%2Fmain%2Fllm-logging%2Fllm_logging_v1.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/apigee-samples/main/llm-logging/llm_logging_v1.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-logging/llm_logging_v1.ipynb">
          <img src="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/images/github-mark.png?raw=true" width="30" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />

# Logging Sample

Logging both prompts and candidate responses from large language models (LLMs) allows for detailed analysis and improvement of the model's performance over time. By examining past interactions, AI practitioners can identify patterns leading to refinements in the training data or model architecture. Furthermore, by examining the prompts security teams can detect malicious intent, such as attempts to extract sensitive information or generate harmful content.

Additionally, logging the generated candidates provides insights into the LLM's behavior and helps identify any biases or vulnerabilities in the model itself. This information can then be used to improve security measures, fine-tune the model, and mitigate potential risks associated with LLM usage.

![image](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-logging/images/llm-logging.png?raw=1)


# Benefits of Logging with Apigee and Google Cloud Logging

* **Seamless logging**: Effortlessly capture prompts, candidate responses, and metadata without complex coding.
* **Scalable and secure**: Leverage Google Cloud's infrastructure for reliable and secure log management.

# How does it work?

1. Prompt request is receved by an Apigee Proxy.
2. Apigee extracts prompt and candidate responses.
3. Apigee logs prompt and candidate responses to Cloud Logging.

# Setup

Use the following GCP CloudShell tutorial. Follow the instructions to deploy the sample.

[![Open in Cloud Shell](https://gstatic.com/cloudssh/images/open-btn.svg)](https://ssh.cloud.google.com/cloudshell/open?cloudshell_git_repo=https://github.com/GoogleCloudPlatform/apigee-samples&cloudshell_git_branch=main&cloudshell_workspace=.&cloudshell_tutorial=llm-logging/docs/cloudshell-tutorial.md)

# Test Sample

## Install dependencies


In [ ]:
!pip install -Uq google-cloud-aiplatform
!pip install -Uq google-genai
!pip install -Uq langchain
!pip install -Uq langchain-google-vertexai

## Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using Vertex AI Workbench or Colab Enterprise.

In [10]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

## Initialize notebook variables

* **PROJECT_ID**: The default GCP project to use when making Vertex API calls.
* **REGION**: The default location to use when making API calls.
* **API_ENDPOINT**:  Desired API endpoint, e.g. https://apigee.iloveapimanagement.com/generate

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}
LOCATION = ""  # @param {type:"string"}
API_ENDPOINT = "https://APIGEE_HOST/v1/samples/llm-logging"  # @param {type:"string"}
MODEL = "gemini-2.0-flash"

## Test Sample

Apigee allows you to seamlessly send logs to Cloud Logging using native integration with the [Message Logging](https://cloud.google.com/apigee/docs/api-platform/reference/policies/message-logging-policy#cloudloggingelement) policy. This sample also includes a message chunking solution that allows logging very long messages (ex. 1M tokens supported by Gemini) and connecting them together using a unique message identifier.

With the following cell you'll be able to invoke an LLM and both prompt and candidate resposne will be logged in Cloud Logging.

### Using GenAI SDK

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    http_options=types.HttpOptions(api_version='v1', base_url=API_ENDPOINT)
)

response = client.models.generate_content(
    model=MODEL, contents='Why is the earth round?'
)
print(response.text)

### Using LangChain SDK

In [ ]:
from langchain_google_vertexai import VertexAI

# Initialize LangChain
model = VertexAI(
      project=PROJECT_ID,
      location=LOCATION,
      api_endpoint=API_ENDPOINT,
      api_transport="rest",
      streaming=False,
      model_name=MODEL)
prompts = ["Provide an explanation about the benefits of using sunscreen. Make sure to make it as long as a novel."]

for prompt in prompts:
  print(model.invoke(prompt))

## Explore and Analyze Logs with Cloud Logging

1. Navigate to the Cloud Logging Explorer by [**clicking here**](https://console.cloud.google.com/logs/query?_ga=2.194228271.307340908.1727018794-898542846.1726863667).

2. Set the query filter. Make sure to replace the `PROJECT_ID` with the Apigee project ID:

  ```
  logName="projects/PROJECT_ID/logs/apigee"
  ```
3. Run the query and explore the logs. See example below:

  ![image](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-logging/images/logs-explorer.png?raw=1)
